# Fierce Research Portfolio Execution

##### Import Packages

In [41]:
import eikon as tr
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime

In [43]:
tr.set_app_key('29e7c9361eb64e748246349dff7c07a532e1f41d')

In [44]:
trading, e= tr.get_data(['HXU.TO','NUGT.K', 'SPY', 'QQQ.O','SPXL.K','TQQQ.O'],['TR.closeprice.date','TR.closeprice','TR.PriceAvg20D','TR.PriceAvg200D'])

##### Import ETFs Prices

In [45]:
trading['Date'] = trading['Date'].apply(pd.to_datetime)
trading.set_index('Date', inplace=True)
trading.columns = map(str.lower, trading.columns)
trading.tail().round()

,instrument,close price,20-day sma,200-day sma
Date,,,,
2020-06-12 00:00:00+00:00,NUGT.K,62.0,70.0,123.0
2020-06-12 00:00:00+00:00,SPY,304.0,305.0,301.0
2020-06-12 00:00:00+00:00,QQQ.O,236.0,234.0,208.0
2020-06-12 00:00:00+00:00,SPXL.K,40.0,41.0,51.0
2020-06-12 00:00:00+00:00,TQQQ.O,85.0,83.0,75.0


##### Set Tactical Asset Allocation Weights For Each ETF

In [46]:
activeportusd = 50000
trading['tactical aa'] = [.05,.05,.20,.20,.25,.25]

In [47]:
trading['maxnotional'] = activeportusd*trading['tactical aa']
trading['num of shrs'] = (trading['maxnotional']/trading['close price']).round()

##### Position Size Guidelines

In [48]:
position_sizes = []
for idx, row in trading.iterrows():
    i = row['close price']
    x = row['20-day sma']
    y = row['200-day sma']
    z = row['num of shrs']
    if (i > x) and (i > y):
        position_sizes.append(1*z)
    elif ((i > x) and (i < y)) or ((i < x) and (i > y)):
        position_sizes.append(0.5*z)
    elif (i < x) and (i < y):
        position_sizes.append(0.1*z)
    else:
        position_sizes.append(np.nan)

trading['position size'] = position_sizes

##### Price Close VS 20D VS 200D Times Number Of Shares

In [49]:
trading.round(2)

,instrument,close price,20-day sma,200-day sma,tactical aa,maxnotional,num of shrs,position size
Date,,,,,,,,
2020-06-12 00:00:00+00:00,HXU.TO,31.66,31.86,37.31,0.05,2500.0,79.0,7.9
2020-06-12 00:00:00+00:00,NUGT.K,62.31,69.93,123.38,0.05,2500.0,40.0,4.0
2020-06-12 00:00:00+00:00,SPY,304.21,304.74,300.98,0.20,10000.0,33.0,16.5
2020-06-12 00:00:00+00:00,QQQ.O,235.88,233.61,207.51,0.20,10000.0,42.0,42.0
2020-06-12 00:00:00+00:00,SPXL.K,39.88,40.84,51.04,0.25,12500.0,313.0,31.3
2020-06-12 00:00:00+00:00,TQQQ.O,84.80,83.40,74.53,0.25,12500.0,147.0,147.0
